In [0]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
from IPython import display
display.clear_output()
from fastai.tabular import *

In [0]:
# import pandas as pd, numpy as np, matplotlib.pyplot as plt
df = pd.read_csv('doctor_train.csv')
test_df = pd.read_csv('doctor_test.csv')

cols = ['age','Money','day','Time','Doctor_visits','last_visit',
        'cured_in','Profession','Status','edu','Irregular','residence',
        'prev_diagnosed','communication','Month','side_effects']
dep_var = ['Y']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_y = le.fit(df['Y'])
df['Y'] = le_y.transform(df['Y'])

In [0]:
df_D = pd.get_dummies(df[cols])
test_df_D = pd.get_dummies(test_df[cols])
df_D=df_D.fillna(0)
# df_D[df_D < 0] = 0
test_df_D=test_df_D.fillna(0)
# test_df_D[test_df_D < 0] = 0

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_D, df['Y'], random_state=45, test_size=0.3)
from imblearn.over_sampling import SMOTE
smt = SMOTE(ratio='auto')
Xtr, ytr = smt.fit_resample(X_train, y_train)
Xtr = pd.DataFrame(Xtr)
Xtr.columns = X_train.columns

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [6]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
clf = LinearDiscriminantAnalysis()
clf.fit(Xtr,ytr)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

In [7]:
y_pred = clf.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8596760846435749
[[6518  973]
 [ 214  754]]
              precision    recall  f1-score   support

           0       0.97      0.87      0.92      7491
           1       0.44      0.78      0.56       968

    accuracy                           0.86      8459
   macro avg       0.70      0.82      0.74      8459
weighted avg       0.91      0.86      0.88      8459



In [0]:
y_pred = clf.predict(test_df_D)
le_y = le.inverse_transform(y_pred)
output = pd.DataFrame({'id': test_df.ID,
                       'Y': le_y})
output.to_csv('submission.csv', index=False)
from google.colab import files
files.download('submission.csv')